In [ ]:
import numpy as np
import pandas as pd
import pickle as pk
import time

In [ ]:
start_time = time.time()

In [ ]:
city = 'Perth'

In [ ]:
data = pd.read_csv("data_process/_RawData/" + city + "/POIs.csv")
quantity_of_POIS = len(data)
quantity_of_POIS

In [ ]:
map_of_POIs = {}
index = 0
for poi in list(data['poiID']):
    map_of_POIs[poi] = index
    index = index + 1

map_of_POIs

In [ ]:
data = pd.read_csv("data_process/_RawData/" + city + "/touristsVisits.csv")
#data = data.head(10)
data.head()


In [ ]:
quant = len(data)
quant

In [ ]:
len(data['userID'].unique())

In [ ]:
global_adj = np.matrix(np.zeros(shape=(quantity_of_POIS,quantity_of_POIS)))

local_adj = dict.fromkeys(list(data['userID']), None)
for item in list(data['userID']):
    local_adj[item] = np.matrix(np.zeros(shape=(quantity_of_POIS,quantity_of_POIS)))
#local_adj = dict.fromkeys(list(data['userID']),np.matrix(np.zeros(shape=(quantity_of_POIS,quantity_of_POIS))))
#local_adj

In [ ]:
#local_adj

In [ ]:
data

In [ ]:
unique_seqID = list(data['seqID'].unique())

In [ ]:
sequences_dict = {}
#biggest_seq = data['seqID'].max()

for seq in unique_seqID:
    sequence = list(data[data['seqID'] == seq]['poiID'].unique())
    id_user = data[data['seqID'] == seq]['userID'].unique()[0]
    
    if id_user in sequences_dict and len(sequence) > len(sequences_dict[id_user]):
        sequences_dict[id_user] = sequence
        
    elif id_user not in sequences_dict:
        sequences_dict[id_user] = sequence
        
    

In [ ]:
sequences_dict

In [ ]:
#sequences_dict
with open("data_process/" + city +  "/sequence","wb") as file:
    pk.dump(sequences_dict, file)

In [ ]:
#data = data.groupby('seqID')
for item in sequences_dict:
    if(len(sequences_dict[item]) > 1):
        for poi in range(0, len(sequences_dict[item])-1):
            user_id = item
            
            i = map_of_POIs[sequences_dict[item][poi]]
            j = map_of_POIs[sequences_dict[item][poi + 1]]
            
            local_adj[user_id][i,j] = local_adj[user_id][i,j] + 1
            local_adj[user_id][j,i] = local_adj[user_id][j,i] + 1

            global_adj[i,j] = global_adj[i,j] + 1
            global_adj[j,i] = global_adj[j,i] + 1 

In [ ]:
#global_adj

In [ ]:
with open("data_process/" + city + "/local","wb") as file:
    pk.dump(local_adj, file)
with open("data_process/" + city + "/global","wb") as file:
    pk.dump(global_adj, file)

In [ ]:
final_time = time.time()

In [ ]:
file = open('data_process/' + city + '/global', 'rb')
d = pk.load(file)

In [ ]:
d

In [ ]:
file = open('data_process/' + city + '/local', 'rb')
d = pk.load(file)

In [ ]:
d

In [ ]:
final_time - start_time

In [ ]:
f = open("results/" + city + "/time", "w")
f.writelines("process_data=" + str(final_time - start_time) + "\n")
f.close()